##Processing Pipline

## - no debug mode

In [ ]:
def process7(input):
 
    # step 1 get edges
    # after_thr=thresholding(input)
    # after_canny=canny(input)
    # combined = combine_threshold(after_thr,after_canny)
    # region_of_inter = region_of_interest(input)

    after_canny=canny(input)
    # plt_images(input,"input",after_canny,"after canny")

    blur = cv2.blur(input,(5,5))

    gradient_combined = apply_thresholds(blur) #input
    # Color thresholding
    s_binary = apply_color_threshold(blur) #input
    # Combine Gradient and Color thresholding
    combined_binary = combine_threshold(s_binary, gradient_combined)

    combined = combine_threshold(after_canny,combined_binary)
    # plt_images(input,"input",after_canny,"after canny")
    
    # step 2 warp canny image
    after_warp1=perspective_warp(combined)
    # after_warp1=strong_warp(after_warp1)
    # plt_images(after_canny,"after canny",after_warp,"after warp")
    after_warp = remove_noise(after_warp1)
    # rect = Draw_rectangle(after_warp)#take 1 channel img
    # # rect_img=cv2.add(rect,after_warp,dtype = cv2.CV_8U)
    # inv_rect =inv_perspective_warp(rect) 

    # step 3 paint area between lanes
    # out_img, window_img, filled_lanes_img, left_curverad, right_curverad
    paint,window_img,filled_lanes_img, left_curverad, right_curverad, center_diff = search_around_poly(after_warp)
    # plt_images(after_warp,"after warp",filled_lanes_img,"filled lanes")
    # plt_images(window_img,"window img",paint," painted area ")


    # step 4 inverse perspective
    frame =inv_perspective_warp(paint, 
                        dst_size=(input.shape[1],input.shape[0]),
                        src=np.float32([(0,0), (1, 0), (0,1), (1,1)]),
                        dst=np.float32([(0.43,0.65),(0.58,0.65),(0.1,1),(1,1)]))


    # step 5 add painting to input image
    result = cv2.addWeighted(frame, 0.3, input, 0.7, 0,dtype = cv2.CV_8U) 
   
    curverad= (left_curverad + right_curverad)/2
    side_pos = 'right'
    if center_diff <= 0:
        side_pos = 'left'
    cv2.putText(result, 'Radius of Curvature='+str(round(curverad,3))+'m ',(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
    cv2.putText(result, 'Vehicle is '+str(abs(round(center_diff,3)))+'m '+side_pos+' of center',(50,100), cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)

    return result

## - debug mode

In [ ]:
def process7_debug(input):
 
    # step 1 get edges
    # after_thr=thresholding(input)
    # after_canny=canny(input)
    # combined = combine_threshold(after_thr,after_canny)
    # region_of_inter = region_of_interest(input)

    after_canny=canny(input)
    # plt_images(input,"input",after_canny,"after canny")

    blur = cv2.blur(input,(5,5))

    gradient_combined = apply_thresholds(blur) #input
    # Color thresholding
    s_binary = apply_color_threshold(blur) #input
    # Combine Gradient and Color thresholding
    combined_binary = combine_threshold(s_binary, gradient_combined)

    combined = combine_threshold(after_canny,combined_binary)
    # plt_images(input,"input",after_canny,"after canny")

    ##############################################
    # show after canny+,  combined_binary,  combined, after warp+, paint, rect, inv_rect, result+

    # step 2 warp canny image
    after_warp1=perspective_warp(combined)
    # after_warp1=strong_warp(after_warp1)
    # plt_images(after_canny,"after canny",after_warp,"after warp")
    after_warp = remove_noise(after_warp1)
    rect = Draw_rectangle(after_warp)#take 1 channel img
    # # rect_img=cv2.add(rect,after_warp,dtype = cv2.CV_8U)
    inv_rect =inv_perspective_warp(rect) 

    # step 3 paint area between lanes
    # out_img, window_img, filled_lanes_img, left_curverad, right_curverad
    paint,window_img,filled_lanes_img, left_curverad, right_curverad, center_diff = search_around_poly(after_warp)
    # plt_images(after_warp,"after warp",filled_lanes_img,"filled lanes")
    # plt_images(window_img,"window img",paint," painted area ")


    # step 4 inverse perspective
    frame =inv_perspective_warp(paint, 
                        dst_size=(input.shape[1],input.shape[0]),
                        src=np.float32([(0,0), (1, 0), (0,1), (1,1)]),
                        dst=np.float32([(0.43,0.65),(0.58,0.65),(0.1,1),(1,1)]))


    # step 5 add painting to input image
    result = cv2.addWeighted(frame, 0.3, input, 0.7, 0,dtype = cv2.CV_8U) 
    # plt_images(input, ' input ', result, ' result ')
    # iny = process_image_rect(input)
    # print("combined: ",combined.shape,combined)
    # print("after_warp: ",after_warp.shape,after_warp)
    curverad= (left_curverad + right_curverad)/2
    side_pos = 'right'
    if center_diff <= 0:
        side_pos = 'left'
    cv2.putText(result, 'Radius of Curvature='+str(round(curverad,3))+'m ',(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
    cv2.putText(result, 'Vehicle is '+str(abs(round(center_diff,3)))+'m '+side_pos+' of center',(50,100), cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
#combine_images( lane_area_img1(resul), edges(binary), warp(binary),warp_rect(rect),warp_highlight(paint), lane(inv_rect))
    out = combine_images(result,combined,after_warp,rect,paint,inv_rect) # after_canny, after_warp,input must be binary
    # cv2_imshow(out)

    return out